In [30]:
import os

In [31]:
%pwd

'c:\\Users\\fatima\\Documents\\FK2\\DataScientist\\Text-Summerizer-Project'

In [3]:
os.chdir('../')

In [16]:
%pwd

'c:\\Users\\fatima\\Documents\\FK2\\DataScientist\\Text-Summerizer-Project'

In [46]:
from dataclasses import dataclass
from pathlib import Path

## An entity is the return data type of a function 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [39]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories


In [40]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [41]:
import os
import  urllib.request as request
import zipfile
from textsummarizer.logging import logger
from textsummarizer.utils.common import get_size

In [54]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        print("self.config.local_data_file", self.config.local_data_file)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [57]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-05 16:59:50,621: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-05 16:59:50,624: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-05 16:59:50,626: INFO: common: created directory at: artifacts]
[2025-03-05 16:59:50,627: INFO: common: created directory at: artifacts/data_ingestion]


[2025-03-05 16:59:53,357: INFO: 2439361658: artifacts/data_ingestion/samsumdata.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5931:2B3A4F:148BC3C:1726459:67C87653
Accept-Ranges: bytes
Date: Wed, 05 Mar 2025 16:05:39 GMT
Via: 1.1 varnish
X-Served-By: cache-par-lfpg1960093-PAR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1741190739.320666,VS0,VE354
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 231e3209ce135a5f6f3b2b5608027b1750c64e4e
Expires: Wed, 05 Mar 2025 16:10:39 GMT
Source-Age: 0

]
self.config.local_data_file 